# Notebook Setup

Don't change the cells in this section... unless you know what you're doing! (it is used to style some of the other cells).

In [1]:
%%html
<style>
.exercise {
    color:#132743;
    background:#AFE4FE;
    padding:10px;
    border-radius:10px;
}
</style>

In [ ]:
!pip install mediapipe

# ACV Part 1: Wrap up project (day 3)

## The Mediapipe Framework

A framework to apply AI / ML pipelines on videos: https://mediapipe.dev/ 

In [2]:
import cv2
import mediapipe as mp

drawing_utils = mp.solutions.drawing_utils
drawing_styles = mp.solutions.drawing_styles

def run_filter_with_mediapipe_model(mediapipe_model, mediapipe_based_filter):
    """Run a media pipe model on each video frame grabbed by the webcam and draw results on it

    Args:
        mediapipe_model (): A mediapipe model
        mediapipe_based_filter (): a function to draw model results on frame

    Returns:
        np.ndarray, mediapipe model result
    """
    cap = cv2.VideoCapture(0)
    
    try:
        with mediapipe_model as model:
            while cap.isOpened():
                success, image = cap.read()

                if not success:
                    print("Ignoring empty camera frame.")
                    continue     # If loading a video, use 'break' instead of 'continue'.

                # Flip the image horizontally for a later selfie-view display, and convert
                # the BGR image to RGB.
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

                try:
                    results = model.process(image)
                except Exception:
                    results = None

                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                if results and results.pose_landmarks:
                    result_image = mediapipe_based_filter(image, results)
                else:
                    result_image = image

                cv2.imshow('MediaPipe', result_image)

                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                    
    finally:
        cap.release()
        cv2.destroyAllWindows()
    
    return image, results

### Holistic model (Hands, Face and Pose tracking)

In [3]:
# see https://google.github.io/mediapipe/solutions/holistic#python-solution-api
Holistic = mp.solutions.holistic.Holistic

def draw_holistic_results(image, results, show_hands=True, show_face=True, show_pose=False):
    if show_hands:
        drawing_utils.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp.solutions.holistic.HAND_CONNECTIONS,
            connection_drawing_spec=drawing_styles.get_default_hand_connections_style()
        )

        drawing_utils.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp.solutions.holistic.HAND_CONNECTIONS,
            connection_drawing_spec=drawing_styles.get_default_hand_connections_style()
        )

    if show_face:
        drawing_utils.draw_landmarks(
            image,
            results.face_landmarks,
            mp.solutions.holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=drawing_utils.DrawingSpec(thickness=0, circle_radius=0, color=(255, 255, 255)),
            connection_drawing_spec=drawing_styles.get_default_face_mesh_contours_style()
        )

    if show_pose:
        drawing_utils.draw_landmarks(
            image,
            results.pose_landmarks,
            mp.solutions.holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=drawing_styles.get_default_pose_landmarks_style()
        )
    
    return image

In [4]:
last_image, last_results = run_filter_with_mediapipe_model(
    mediapipe_model=Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5),
    mediapipe_based_filter=draw_holistic_results
)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Create your Photo Booth application!

<div class="exercise">
<h4>Now you have all you need to develop your own photobooth application!</h4>

Here are some ideas to get you going:

<ul>
    <li>Add props to the image: a mask, glasses, little characters on some or all of your fingers <em>(align the props to the detected keypoints for the best effect!)</em></li>
    <li>Modify your appearance (add some gloss or lipstick, change your eye color, create a cartoon version of yourself. ...)</li>
    <li>Apply some "Instagram-like" filters to your webcam stream (sepia, Xray, thermal camera, beautify ("blurify"), ...)</li>
    <li>Maybe a bit more advanced but... Can you execute a Face Swap ?!</li>
    <li>Just unleash your endless creativity!</li>
</ul>